In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained("unsloth/gemma-2-9b-it", use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    "unsloth/gemma-2-9b-it",
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16
)

text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=64
)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [2]:
from langchain_huggingface import HuggingFaceEmbeddings


embedding_name = 'sbunlp/fabert'
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_name
)

No sentence-transformers model found with name sbunlp/fabert. Creating a new one with mean pooling.
Some weights of BertModel were not initialized from the model checkpoint at sbunlp/fabert and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from langchain_huggingface.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=text_gen)

def apply_chat_template_and_response(prompt):
    messages = [
    {'role': 'user', 'content': prompt}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    return llm.invoke(text).replace(text, '')

In [4]:
print(apply_chat_template_and_response("اسمت چیه؟"))

من Gemma هستم، یک مدل زبان بزرگ باز که توسط Google DeepMind آموزش داده شده‌ام.



In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
response_from_model = apply_chat_template_and_response("Give me an inspirational quote in persian")
parsed_response = parser.parse(response_from_model)
print(parsed_response)

"هر کس تو را به پایین نگاه می برد، تو را به بالا بکش."

**Translation:** "Whoever looks down on you, pull them up."


This quote emphasizes self-belief and resilience. It encourages you to rise above negativity and help others along the way. 




In [6]:
from langchain_core.prompts import PromptTemplate

template = """
You are a helpful and knowledgeable AI assistant. Use only the information retrieved from the documents to answer the user's question in Persian (Farsi). If the answer is not found in the retrieved context, respond with: "متاسفانه اطلاعاتی در این مورد ندارم." Do not use your own knowledge beyond the provided context. Be accurate, clear, and polite. Never mention the documents or the retrieval process in your response. Just respond naturally in Persian.
Context: {context}

Question: {question}

Answer:

"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nYou are a helpful and knowledgeable AI assistant. Use only the information retrieved from the documents to answer the user\'s question in Persian (Farsi). If the answer is not found in the retrieved context, respond with: "متاسفانه اطلاعاتی در این مورد ندارم." Do not use your own knowledge beyond the provided context. Be accurate, clear, and polite. Never mention the documents or the retrieval process in your response. Just respond naturally in Persian.\nContext: Here is some context\n\nQuestion: Here is a question\n\nAnswer:\n\n'

In [7]:
context = "من علیرضا هستم و تهران زندگی میکنم."
formatted_prompt = prompt.format(context = context, question="اسم من چیه؟")
response_from_model = apply_chat_template_and_response(formatted_prompt)
parsed_response = parser.parse(response_from_model)
print(parsed_response.replace(formatted_prompt, ""))

اسم تو علیرضا هست.  



In [8]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("my_pdf.pdf")
pages = loader.load_and_split()

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
text_documents = text_splitter.split_documents(pages)

In [10]:
text_documents

[Document(metadata={'producer': 'Microsoft® Word LTSC', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2025-12-30T11:13:06+03:30', 'author': 'myuser', 'moddate': '2025-12-30T11:13:06+03:30', 'source': 'my_pdf.pdf', 'total_pages': 53, 'page': 0, 'page_label': '1'}, page_content='1-مقدمه ..................................................................................................... 2 \n2- گام اول: درک خواسته های   کسب و کار یا Business Understanding ................... 5 \n3- گام دوم: درک داده هایا  Data Understanding .............................................. 7 \n4- گام سوم: آماده سازی   داده ها  یا  Data Preparation ........................................ 11 \n5- گام چهارم: مدل سازی  یا  Modeling ............................................................ 17 \n6- گام پنجم: ارزیابی  یا    Evaluating ................................................................ 44 \n7- گام ششم: بکارگیری   یا   Deployment .......................................................... 53'),

In [11]:
from langchain_community.vectorstores import DocArrayInMemorySearch
vectorstore = DocArrayInMemorySearch.from_documents(text_documents, embedding=embeddings)

In [20]:
questions = [
    "چند مورد از خسارت های ناشی از مشکل امنیتی در سازمان ها را بیان کنید؟",
    "دیتاست NSL-KDDدر چه سالی عرضه شد؟",
    "دیتاست NSL-KDD شامل چند نمونه ترافیک حمله و چند نمونه ترافیک عادی است؟",
    "مدل با استفاده از الگوریتم جنگل تصادفی به چه دقتی  بر روی داده های تست دست یافته است؟",
    "مدل با استفاده از الگوریتم KNN به چه دقتی بر روی داده های تست دست یافته است؟",
    "یکی از مهم ترین نتایج مدل سازی هنگام تست مدل های طبقه بندی دودویی مشاهده شد چیست؟",
    "یکی از مشکلاتی که در دسته بندی چند کلاسه مشاهده کردیم چیست؟",
    "الگوریتم جنگل تصادفی را توضیح بده؟"
]

for question in questions:
    retriever = vectorstore.as_retriever()
    retrieved_context = retriever.invoke(question)
    formatted_prompt = prompt.format(context=retrieved_context, question=question)
    response_from_model = apply_chat_template_and_response(formatted_prompt)
    parsed_response = parser.parse(response_from_model)

    print(f"Question: {question}")
    print(f"Answer: {parsed_response}")
    print()

Question: چند مورد از خسارت های ناشی از مشکل امنیتی در سازمان ها را بیان کنید؟
Answer: کاهش اعتماد مشتریان، سرمایه گذاران و کارمندان، کاهش چشمگیر مشتریان سازمان، از دست رفتن اعتبار سازمان در میان رقبا، افزایش هزینه های سازمانی و کاهش سطح بهره وری و درآمد سازمان. 




Question: دیتاست NSL-KDDدر چه سالی عرضه شد؟
Answer: دیتاست NSL-KDD در سال 2009 عرضه شد. 


Question: دیتاست NSL-KDD شامل چند نمونه ترافیک حمله و چند نمونه ترافیک عادی است؟
Answer: دیتاست NSL-KDD  شامل 58630 مورد ترافیک حمله و 67343 نمونه ترافیک عادی است.  


Question: مدل با استفاده از الگوریتم جنگل تصادفی به چه دقتی  بر روی داده های تست دست یافته است؟
Answer: دقت مدل ما بر روی داده های تست  0.8035753892560884  می باشد. 


Question: مدل با استفاده از الگوریتم KNN به چه دقتی بر روی داده های تست دست یافته است؟
Answer: دقت مدل ما بر روی داده های تست 0.8014904848511734 می باشد.  


Question: یکی از مهم ترین نتایج مدل سازی هنگام تست مدل های طبقه بندی دودویی مشاهده شد چیست؟
Answer: یکی از مهم ترین نتایج مدل سازی هنگام تست مدل ها

In [21]:
new_embedding_name = 'heydariAI/persian-embeddings'
new_embeddings = HuggingFaceEmbeddings(
    model_name=new_embedding_name
)

In [22]:
new_vectorstore = DocArrayInMemorySearch.from_documents(text_documents, embedding=new_embeddings)

In [23]:
questions = [
    "چند مورد از خسارت های ناشی از مشکل امنیتی در سازمان ها را بیان کنید؟",
    "دیتاست NSL-KDDدر چه سالی عرضه شد؟",
    "دیتاست NSL-KDD شامل چند نمونه ترافیک حمله و چند نمونه ترافیک عادی است؟",
    "مدل با استفاده از الگوریتم جنگل تصادفی به چه دقتی  بر روی داده های تست دست یافته است؟",
    "مدل با استفاده از الگوریتم KNN به چه دقتی بر روی داده های تست دست یافته است؟",
    "یکی از مهم ترین نتایج مدل سازی هنگام تست مدل های طبقه بندی دودویی مشاهده شد چیست؟",
    "یکی از مشکلاتی که در دسته بندی چند کلاسه مشاهده کردیم چیست؟",
    "الگوریتم جنگل تصادفی را توضیح بده؟"
]

for question in questions:
    retriever = new_vectorstore.as_retriever()
    retrieved_context = retriever.invoke(question)
    formatted_prompt = prompt.format(context=retrieved_context, question=question)
    response_from_model = apply_chat_template_and_response(formatted_prompt)
    parsed_response = parser.parse(response_from_model)

    print(f"Question: {question}")
    print(f"Answer: {parsed_response}")
    print()

Question: چند مورد از خسارت های ناشی از مشکل امنیتی در سازمان ها را بیان کنید؟
Answer: کاهش اعتماد مشتریان، سرمایه گذاران و کارمندان، کاهش چشمگیر مشتریان سازمان، از دست رفتن اعتبار سازمان در میان رقبا و افزایش هزینه های سازمانی. 


Question: دیتاست NSL-KDDدر چه سالی عرضه شد؟
Answer: دیتاست NSL-KDD در سال 2009 عرضه شد. 




Question: دیتاست NSL-KDD شامل چند نمونه ترافیک حمله و چند نمونه ترافیک عادی است؟
Answer: دیتاست NSL-KDD شامل 58630 نمونه ترافیک حمله و 67343 نمونه ترافیک عادی است.  


Question: مدل با استفاده از الگوریتم جنگل تصادفی به چه دقتی  بر روی داده های تست دست یافته است؟
Answer: دقت مدل ما بر روی داده های تست برابر با 0.8035753892560884 است.  


Question: مدل با استفاده از الگوریتم KNN به چه دقتی بر روی داده های تست دست یافته است؟
Answer: دقت مدل بر روی داده های تست 0.8014904848511734 است.  


Question: یکی از مهم ترین نتایج مدل سازی هنگام تست مدل های طبقه بندی دودویی مشاهده شد چیست؟
Answer: همانطور که ملاحظه می کنید، دقت مدل ما بر روی داده های تست افت قابل ملاحظه ای داشته ا